In [22]:
import numpy as np
import pandas as pd
import lightgbm as lgb

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification

In [23]:
from IPython.display import set_matplotlib_formats
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
import pandas as pd

%matplotlib inline
set_matplotlib_formats('svg')

DATA LOADING

In [24]:
df_train = pd.read_csv('../data/diamonds_train.csv')
df_test = pd.read_csv('../data/diamonds_predict.csv')

In [25]:
X, y = make_classification(n_features=3, n_redundant=0, n_informative=2,
                           random_state=42, n_clusters_per_class=1, n_samples=5000)

In [26]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [28]:
target = 'price'
cat_features = ['cut', 'color', 'clarity']
num_features = ['carat', 'depth','table']
features = num_features + cat_features

TRAIN

In [ ]:
model = lgb.LGBMClassifier(num_leaves=40,
                        learning_rate=0.05,
                        n_estimators=128,
                        max_depth=16,
                        n_jobs=-1)
model.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='l2',
        early_stopping_rounds=5)

model.fit(df_train[num_features], df_train[target])

[1]	valid_0's l2: 0.228447	valid_0's binary_logloss: 0.65
Training until validation scores don't improve for 5 rounds
[2]	valid_0's l2: 0.208772	valid_0's binary_logloss: 0.610437
[3]	valid_0's l2: 0.191047	valid_0's binary_logloss: 0.57449
[4]	valid_0's l2: 0.175113	valid_0's binary_logloss: 0.541788
[5]	valid_0's l2: 0.160724	valid_0's binary_logloss: 0.511819
[6]	valid_0's l2: 0.147776	valid_0's binary_logloss: 0.484374
[7]	valid_0's l2: 0.136103	valid_0's binary_logloss: 0.459127
[8]	valid_0's l2: 0.125523	valid_0's binary_logloss: 0.435743
[9]	valid_0's l2: 0.116059	valid_0's binary_logloss: 0.414318
[10]	valid_0's l2: 0.107441	valid_0's binary_logloss: 0.394307
[11]	valid_0's l2: 0.0997344	valid_0's binary_logloss: 0.375906
[12]	valid_0's l2: 0.0927159	valid_0's binary_logloss: 0.35869
[13]	valid_0's l2: 0.0863946	valid_0's binary_logloss: 0.342721
[14]	valid_0's l2: 0.080675	valid_0's binary_logloss: 0.327813
[15]	valid_0's l2: 0.075513	valid_0's binary_logloss: 0.313925
[16]	va

PREDICTION

In [ ]:
#pred
y_pred = model.predict(X_test, num_iteration=model.best_iteration_)
# eval
print('The RMSE of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

In [19]:
def rmsle(y_true, y_pred):
    return 'RMSLE', np.sqrt(np.mean(np.power(np.log1p(y_pred) - np.log1p(y_true), 2))), False

In [20]:
model.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='RMSLE',
        early_stopping_rounds=5)

LightGBMError: Number of classes should be specified and greater than 1 for multiclass training

In [ ]:
def rae(y_true, y_pred):
    return 'RAE', np.sum(np.abs(y_pred - y_true)) / np.sum(np.abs(np.mean(y_true) - y_true)), False

PREDICTIONS

In [ ]:
#pred
y_pred = model.predict(X_test, num_iteration=model.best_iteration_)
# eval
print('The rmsle of prediction is:', rmsle(y_test, y_pred)[1])
print('The rae of prediction is:', rae(y_test, y_pred)[1])

In [ ]:
y_pred = model.predict(df_test[features])

In [ ]:
df_test['num_features'] = y_pred

In [ ]:
submission_v3_1609 = pd.DataFrame({'price': y_pred})

In [ ]:
submission_v3_1609.index.name='id'

In [ ]:
submission_v3_1609.head()

In [ ]:
submission_v3_1609.describe()

In [ ]:
#submission_v3_1609.price.clip(0, 20000, inplace=True)

In [ ]:
#submission_v3_1609.to_csv('submission_v3_1609.csv')